<a href="https://colab.research.google.com/github/arshpreetw11/UCI-HAR-ANN-Model/blob/main/proj4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip archive.zip

Archive:  archive.zip
  inflating: UCI-HAR Dataset/README.txt  
  inflating: UCI-HAR Dataset/activity_labels.txt  
  inflating: UCI-HAR Dataset/features.txt  
  inflating: UCI-HAR Dataset/features_info.txt  
  inflating: UCI-HAR Dataset/test/Inertial Signals/body_acc_x_test.txt  
  inflating: UCI-HAR Dataset/test/Inertial Signals/body_acc_y_test.txt  
  inflating: UCI-HAR Dataset/test/Inertial Signals/body_acc_z_test.txt  
  inflating: UCI-HAR Dataset/test/Inertial Signals/body_gyro_x_test.txt  
  inflating: UCI-HAR Dataset/test/Inertial Signals/body_gyro_y_test.txt  
  inflating: UCI-HAR Dataset/test/Inertial Signals/body_gyro_z_test.txt  
  inflating: UCI-HAR Dataset/test/Inertial Signals/total_acc_x_test.txt  
  inflating: UCI-HAR Dataset/test/Inertial Signals/total_acc_y_test.txt  
  inflating: UCI-HAR Dataset/test/Inertial Signals/total_acc_z_test.txt  
  inflating: UCI-HAR Dataset/test/X_test.txt  
  inflating: UCI-HAR Dataset/test/subject_test.txt  
  inflating: UCI-HAR Dataset/

In [2]:
!ls

 archive.zip   sample_data  'UCI-HAR Dataset'


In [3]:
!ls 'UCI-HAR Dataset'

activity_labels.txt  features_info.txt	features.txt  README.txt  test	train


In [4]:
import numpy as np
import pandas as pd

In [5]:
X_train=np.loadtxt('UCI-HAR Dataset/train/X_train.txt')
y_train=np.loadtxt('UCI-HAR Dataset/train/y_train.txt')
X_test=np.loadtxt('UCI-HAR Dataset/test/X_test.txt')
y_test=np.loadtxt('UCI-HAR Dataset/test/y_test.txt')

In [7]:
np.unique(y_train)#to use sparsecategorical_crossentropy we should get zero indexed array

array([1., 2., 3., 4., 5., 6.])

In [8]:
y_train=y_train.astype(int)-1
y_test=y_test.astype(int)-1

In [9]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5])

In [11]:
X_train.shape

(7352, 561)

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, PReLU, Input
from tensorflow.keras.optimizers import Adam, RMSprop

def build_model(hp):
    model = Sequential()

    # Input layer
    model.add(Input(shape=(X_train.shape[1],)))

    # Optimizer + LR
    optimizer_name = hp.Choice("optimizer", ["adam", "rmsprop"])
    lr = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])

    if optimizer_name == "adam":
        optimizer = Adam(learning_rate=lr)
    else:
        optimizer = RMSprop(learning_rate=lr)

    # Hidden layers
    for i in range(hp.Int("num_hidden_layers", min_value=3, max_value=6)):

        units = hp.Int(f"units_{i}", min_value=16, max_value=256, step=16)
        activation = hp.Choice(f"activation_{i}", ["relu", "prelu", "tanh"])

        model.add(Dense(units))

        if activation == "prelu":
            model.add(PReLU())
        else:
            model.add(Activation(activation))

        model.add(BatchNormalization())

        model.add(
            Dropout(
                rate=hp.Float(
                    f"dropout_{i}",
                    min_value=0.1,
                    max_value=0.5,
                    step=0.1
                )
            )
        )

    # Output layer (binary classification)
    model.add(Dense(6, activation="softmax"))

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [35]:
from tensorflow.keras.callbacks import EarlyStopping

In [36]:
!pip install keras_tuner
import keras_tuner as kt

In [37]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=10
                      ,directory="tuner_logs",project_name="har_datset")

Reloading Tuner from tuner_logs/har_datset/tuner0.json


In [38]:
call=EarlyStopping(monitor='val_accuracy',patience=5,min_delta=0.01,restore_best_weights=True)

In [39]:
tuner.search(X_train,y_train,
             validation_data=(X_test,y_test),epochs=20,batch_size=32,callbacks=[call])

Trial 10 Complete [00h 00m 24s]
val_accuracy: 0.9043094515800476

Best val_accuracy So Far: 0.9602985978126526
Total elapsed time: 00h 04m 40s


In [45]:
best_model=tuner.get_best_models(num_models=1)[0]

best_model.fit(X_train,y_train,epochs=100,batch_size=32,validation_data=(X_test,y_test),callbacks=[call])

Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9391 - loss: 0.1666 - val_accuracy: 0.9498 - val_loss: 0.1564
Epoch 2/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9437 - loss: 0.1624 - val_accuracy: 0.9253 - val_loss: 0.1875
Epoch 3/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9402 - loss: 0.1783 - val_accuracy: 0.8921 - val_loss: 0.2972
Epoch 4/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9397 - loss: 0.1629 - val_accuracy: 0.9528 - val_loss: 0.1389
Epoch 5/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9335 - loss: 0.1754 - val_accuracy: 0.9454 - val_loss: 0.1585
Epoch 6/100
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9400 - loss: 0.1691 - val_accuracy: 0.9399 - val_loss: 0.1992


In [46]:
test_loss,test_acc=best_model.evaluate(X_test,y_test,verbose=0)

In [47]:
test_loss

0.1563858538866043

In [48]:
test_acc

0.9497794508934021